In [ ]:
zip_name = "stations2.zip"

In [ ]:
# SIMPLE
#import urllib2
#response = urllib2.urlopen('http://www.example.com/')
#html = response.read()

# WITH PROGRESS

import urllib2

url = "http://www.radiosure.com/rsdbms/stations2.zip?updated"

#file_name = url.split('/')[-1]
zip_name = "stations2.zip"
u = urllib2.urlopen(url)
f = open(zip_name, 'wb')
meta = u.info()
file_size = int(meta.getheaders("Content-Length")[0])
print "Downloading: %s Bytes: %s" % (zip_name, file_size)

file_size_dl = 0
# block_sz = 262144
block_sz = file_size // 10 + 9
while True:
    buffer = u.read(block_sz)
    if not buffer:
        break

    file_size_dl += len(buffer)
    f.write(buffer)
    status = r"%10d  [%3.2f%%]" % (file_size_dl, file_size_dl * 100. / file_size)
    status = status + chr(8)*(len(status)+1)
    print status + '\n'

f.close()

In [17]:
import zipfile

zip_name = "stations2.zip"

fh = open(zip_name, 'rb')
z = zipfile.ZipFile(fh)
for name in z.namelist():
    outfile = open(name, 'wb')
    outfile.write(z.read(name))
    stations_name = name
    outfile.close()
fh.close()
print stations_name

stations-2016-05-01.rsd


In [18]:
stations = []
with open(stations_name,'r') as fp:
    for line in fp:
        station = line.decode("utf-8", errors = 'replace').strip().split("\t")
        stations.append(station)



In [19]:
stations[0]

[u'\ufeffWeFunk',
 u'Raw uncut deedp funk, quality underground hip-hop, rare grooves on the weekly show from Professor Groove & DJ Static. (www.wefunkradio.com)',
 u'Rap/Hip Hop',
 u'Canada',
 u'English',
 u'http://www.wefunkradio.com/play/shoutcast.pls',
 u'http://www.wefunkradio.com/play/shoutcast.m3u',
 u'http://207.200.96.232:8018',
 u'http://129.173.67.99:8000',
 u'http://129.173.67.99:8000/listen.pls',
 u'http://205.188.215.229:8018']

In [26]:
stat_dict_list = []
first_line = True
for item in stations:
    stat_dict={}
    if first_line:
        st_name = item[0][1:]
        first_line = False
    else:
        st_name = item[0]
    stat_dict['name'] =  st_name
    stat_dict['desc']    = item[1]
    stat_dict['genre']   = item[2]
    stat_dict['country'] = item[3]
    stat_dict['lang']    = item[4]
    stat_dict['links']   = []
    for i in range(5,11):
        if item[i] != '-':
            stat_dict['links'].append(item[i])
    stat_dict_list.append(stat_dict)
    
print len(stat_dict_list)
    
print stat_dict_list[0]

32895
{'lang': u'English', 'name': u'WeFunk', 'links': [u'http://www.wefunkradio.com/play/shoutcast.pls', u'http://www.wefunkradio.com/play/shoutcast.m3u', u'http://207.200.96.232:8018', u'http://129.173.67.99:8000', u'http://129.173.67.99:8000/listen.pls', u'http://205.188.215.229:8018'], 'country': u'Canada', 'genre': u'Rap/Hip Hop', 'desc': u'Raw uncut deedp funk, quality underground hip-hop, rare grooves on the weekly show from Professor Groove & DJ Static. (www.wefunkradio.com)'}


In [27]:
# genres
lines_out = []
for item in stat_dict_list:
    if item['name'].lower().find('nostalgia.') > -1 and item['genre'].lower().find('talk') == -1 and \
       item['genre'].lower().find('public') == -1:
        lines_out.append(item['name'].ljust(60) + item['genre'].ljust(40) + item['country'])
lines_out.sort()
print len(lines_out)
#for item in lines_out:
#    print item

81


In [ ]:
ext_lst = []

for item in stat_dict_list:
    for link in item['links']:
        ext = link[-3:]
        if (ext == unicode('wpl')) \
            or (ext == unicode('ram')):
            ext_lst.append(link)
        

print ext_lst
        

In [44]:
genre_set = set({})
for item in stat_dict_list:
    genre_set.add(item['genre'])
genre_lst = list(genre_set)

genre_lst.sort()

#genre_lst

In [50]:
def get_links(link):
    streams = []
    ext = link[-3:]
    if   ext == 'pls': link = '#PLS:' + link
    elif ext == 'm3u': link = '#M3U:' + link
    elif ext == 'asx': link = '#ASX:' + link
    elif ext == 'wpl': link = '#WPL:' + link
    elif ext == 'ram': link = '#RAM:' + link
    streams.append(link)
    return streams



base = 'stations/'
for item in stat_dict_list:
    #print item
    pre1 = item['genre'][:4]
    pre2 = item['genre'][:6]
    pre_good = (pre1 != 'Talk') and (pre1 != 'All ') and \
               (pre2 != 'Childr') and (pre2 != 'Islami') and (pre2 != 'Religi') and (pre2 != 'Bollyw')
    if pre_good:
        fold_1 = item['genre'].replace('"', "'").replace('/', '-').replace(':', '.').replace('|', '-').replace('*', '-')\
                .replace('<', '-').replace('>', '-').replace('?', '-').replace('\\', '-')
        fold_2 = item['lang']
        dir_path = base + fold_1 + '/' + fold_2
#---------------------------------------------------------        
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
        file_name = item['name'].replace('"', "'").replace('/', '-').replace(':', '.').replace('|', '-').replace('*', '-')\
                    .replace('<', '-').replace('>', '-').replace('?', '-').replace('\\', '-')
        file_path = dir_path + '/' + file_name + '.m3u'
        # print file_path
        f = open(file_path, 'w')
        f.write('#EXTM3U' + '\n')
        f.write('#EXTINF:-1,' + item['name'].encode('utf-8') + '\n')
        for link in item['links']:
            streams = get_links(link)
            for stream in streams:
                f.write(stream.encode('utf-8') + '\n')
        f.write('#GENRE:' + item['genre'].encode('utf-8') + '\n')
        f.write('#COUNTRY:' + item['country'].encode('utf-8') + '\n')
        f.write('#LANGUAGE:' + item['lang'].encode('utf-8') + '\n')
        f.write('#DESCRIPTION:' + item['desc'].encode('utf-8') + '\n')

        f.close
    

In [15]:
f.close()